In [1]:
import numpy as np
from keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from google.colab import drive

drive.mount('/content/drive')

# 분류 가능하게 이미지 가공
    # img1 = image.load_img('/content/drive/My Drive/input/test4.jpg'.format(test_img), target_size=(64, 64))
    # img = image.img_to_array(img1)
    # img = img/255
    # img = np.expand_dims(img, axis=0)

# -*- coding: utf-8 -*-
"""
Created on Wed Jan 20 18:05:48 2021

@author: Owner
"""
import cv2, sys
import numpy as np
from matplotlib import pyplot as plt


imageFile='/content/drive/My Drive/input/test6.jpg'
img = cv2.imread(imageFile, cv2.IMREAD_COLOR)



blur = cv2.GaussianBlur(img, ksize=(3,3), sigmaX=0)
ret, thresh1 = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY)
edged = cv2.Canny(blur, 10, 250)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)



contours, _ = cv2.findContours(closed.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
total = 0

contours_xy = np.array(contours)
contours_xy.shape

x_min, x_max = 0,0
value = list()
for i in range(len(contours_xy)):
    for j in range(len(contours_xy[i])):
        value.append(contours_xy[i][j][0][0]) #네번째 괄호가 0일때 x의 값
        x_min = min(value)
        x_max = max(value)
print(x_min)
print(x_max)
 
# y의 min과 max 찾기
y_min, y_max = 0,0
value = list()
for i in range(len(contours_xy)):
    for j in range(len(contours_xy[i])):
        value.append(contours_xy[i][j][0][1]) #네번째 괄호가 0일때 x의 값
        y_min = min(value)
        y_max = max(value)
print(y_min)
print(y_max)

# image trim 하기
x = x_min
y = y_min
w = x_max-x_min
h = y_max-y_min
img_trim = img[y:y+h, x:x+w]
cv2.imwrite('org_trim.jpg', img_trim)
org_image = cv2.imread('org_trim.jpg')




gray = cv2.cvtColor(org_image, cv2.COLOR_BGR2GRAY)





ret, wafer = cv2.threshold(gray, 254, 1, cv2.THRESH_BINARY_INV)


ret, defect = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY_INV)

ret, defect2 = cv2.threshold(defect, 0, 4, cv2.THRESH_BINARY)


plt.imshow(wafer, cmap="gray"), plt.axis("off") # 이미지를 출력
plt.show()
plt.imshow(wafer)
print(wafer)
plt.imshow(defect, cmap="gray"), plt.axis("off") # 이미지를 출력
plt.show()
plt.imshow(defect)
print(defect)
plt.imshow(defect2, cmap="gray"), plt.axis("off") # 이미지를 출력
plt.show()
plt.imshow(defect2)
print(defect2)


img_height1, img_width1 = wafer.shape


newImage1 = np.zeros( (26,26), dtype = wafer.dtype ) 


new_height1 = img_height1//26 
new_width1 = img_width1//26  

for j in range(26):
    for i in range(26):
        y = j*new_height1
        x = i*new_width1
        pixel = wafer[y:y+new_height1, x:x+new_width1,]  
        newImage1[j,i,] = pixel.sum(dtype='int64')//(new_height1*new_width1) 
        
        
        
        
img_height2, img_width2 = defect2.shape
        
newImage2 = np.zeros( (26,26), dtype = defect2.dtype ) 


new_height2 = img_height2//26 
new_width2 = img_width2//26  

for j in range(26):
    for i in range(26):
        y = j*new_height2
        x = i*new_width2
        pixel = defect2[y:y+new_height2, x:x+new_width2]  
        newImage2[j,i] = pixel.sum(dtype='int64')//(new_height2*new_width2) 
        
kk= newImage1 + newImage2

kk = np.where(kk>2,2,kk)

kk1 = np.where(kk==0,0,0)  # 1층 0으로 공사
kk2 = np.where(kk>=1,1,0)  # 2층 웨이퍼 부분이랑 웨이퍼에서 defect 없는 부분을 1로 나머지는 0으로
kk3 = np.where(kk>=2,1,0)  # 3층 웨이퍼에서 defect 있는 부분만 2로 나머지는 0으로

newkk = np.dstack([kk1,kk2,kk3])

#kk=np.reshape(kk, (1,26, 26))

#newkk = np.zeros( (26,26,3), dtype = kk.dtype ) 
#for j in range(26):
#    for i in range(26):
#        for k in range(26):
#            if kk[]==0 :
#                newkk[i,j,k]= 





## 여기에다가 새로 차원 맞추는 코드 넣으면 됩니다

plt.imshow(kk)


newkk = np.expand_dims(newkk, axis=0)



print(newkk)
print(newkk.shape)
print(type(newkk))


Mounted at /content/drive


error: ignored

In [ ]:
def class_img(test_img):
    
    # 모델 불러오기
    model = load_model('/content/drive/My Drive/model_file_simpleCNN/checkpoint_CNN.h5')

    # 각 레이블의 확률을 numpy배열로 출력
    class_df = model.predict(test_img)

    if(np.argmax(class_df[0]) == 0):
        defect = "Center"

    elif(np.argmax(class_df[0]) == 1):
        defect = "Donut"

    elif(np.argmax(class_df[0]) == 2):
        defect = "Edge-Loc"

    elif(np.argmax(class_df[0]) == 3):
        defect = "Edge-Ring"

    elif(np.argmax(class_df[0]) == 4):
        defect = "Loc"

    elif(np.argmax(class_df[0]) == 5):
        defect = "Near-Full"

    elif(np.argmax(class_df[0]) == 6):
        defect = "Random"

    elif(np.argmax(class_df[0]) == 7):
        defect = "Scratch"

    elif(np.argmax(class_df[0]) == 8):
        defect = "None"

    ## 분류 결과
    # 결과 출력
    print(class_df[0]*100)
    print("Defect: {} ({}%)".format(defect, np.round_(np.max(class_df[0])*100, 2)))

    # 결과 이미지 출력
    # value = "{}: {}%".format(defect, np.round_(np.max(class_df[0])*100, 2))
    # plt.text(20, 62, value, color='red', fontsize=15, bbox=dict(facecolor='white', alpha=0.8))
    # plt.imshow(img1)
    # plt.show()

# print(img.shape)



# 분류!!!
class_img(newkk)